In [11]:
import math
from queue import PriorityQueue
import time

In [12]:
puzzle = 8
dim = int(math.sqrt(puzzle)) + 1
print(dim)

3


In [13]:
goal_state = []
co_ordinates = {}
for i in range(dim):
  temp = []
  for j in range(dim):
    val = i*dim+(j+1)
    temp.append(val)
    co_ord = [i, j]
    co_ordinates[val] = co_ord
  goal_state.append(temp)

last_val = goal_state[-1][-1]
goal_state[-1][-1] = 0
temp_co_ord = co_ordinates[last_val]
del co_ordinates[last_val]
co_ordinates[0] = temp_co_ord
print(last_val)
print(goal_state)
print(co_ordinates)

9
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
{1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [1, 0], 5: [1, 1], 6: [1, 2], 7: [2, 0], 8: [2, 1], 0: [2, 2]}


In [14]:
def manhattan_distance(state):
  m_dist = 0
  for i in range(dim):
    for j in range(dim):
      val = state[i][j]
      if(val == 0):
        continue
      goal_co_ord = co_ordinates[val]
      i_goal = goal_co_ord[0]
      j_goal = goal_co_ord[1]
      m_dist += abs(i_goal - i) + abs(j_goal - j)
  return m_dist

In [15]:
state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
dist = manhattan_distance(state)
print(dist)

14


In [16]:
def misplaced_tile(state):
  m_tile = 0
  for i in range(dim):
    for j in range(dim):
      val = goal_state[i][j]
      if(val == 0):
        continue
      if(val != state[i][j]):
        m_tile += 1
  return m_tile

In [17]:
state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
dist = misplaced_tile(state)
print(dist)

6


In [23]:
class Node:
  def __init__(self, state, parent=None, g=0):
    self.state = state
    self.parent = parent
    self.g = g
    self.h = 0
    self.total_cost = 0
    
  def __lt__(self, other_node):
        return self.total_cost < other_node.total_cost

In [24]:
def expand(node):
  state = node.state
  expanded_nodes = []
  blank_i = dim - 1
  blank_j = dim - 1

  for i in range(dim):
    for j in range(dim):
      if(state[i][j] == 0):
        blank_i = i
        blank_j = j
        break
  if(blank_i > 0):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i - 1][blank_j]
    child[blank_i][blank_j] = val
    child[blank_i - 1][blank_j] = 0
    child_node = Node(child, state, node.g+1)
    expanded_nodes.append(child_node)

  if(blank_i < dim - 1):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i + 1][blank_j]
    child[blank_i][blank_j] = val
    child[blank_i + 1][blank_j] = 0
    child_node = Node(child, state, node.g+1)
    expanded_nodes.append(child_node)

  if(blank_j > 0):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i][blank_j - 1]
    child[blank_i][blank_j] = val
    child[blank_i][blank_j - 1] = 0
    child_node = Node(child, state, node.g+1)
    expanded_nodes.append(child_node)

  if(blank_j < dim - 1):
    child = []
    for lst in state:
      child.append(lst[:])
    val = state[blank_i][blank_j + 1]
    child[blank_i][blank_j] = val
    child[blank_i][blank_j + 1] = 0
    child_node = Node(child, state, node.g+1)
    expanded_nodes.append(child_node)

  return expanded_nodes

In [25]:
state = [[1, 6, 7], [5, 0, 3], [4, 8, 2]]
node = Node(state)
exp_nodes = expand(node)
for node in exp_nodes:
  print(node.state)

[[1, 0, 7], [5, 6, 3], [4, 8, 2]]
[[1, 6, 7], [5, 8, 3], [4, 0, 2]]
[[1, 6, 7], [0, 5, 3], [4, 8, 2]]
[[1, 6, 7], [5, 3, 0], [4, 8, 2]]


In [26]:
def a_star_with_UC(node):
  q = PriorityQueue()
  q.put(node)
  is_visited = set()
  number_of_expansion = 0
  s_time = time.process_time()
  max_queue_size = 0
  while (q.empty() == False):
    nde = q.get()
    print(nde.state, 'is the best sub-puzzle to be solved with g(n): '+str(nde.g))
    if(misplaced_tile(nde.state) == 0):
      e_time = time.process_time()
      print("reached the goal state!")
      print("the number of expansion required: "+ str(number_of_expansion))
      print("maximum queue size: "+ str(max_queue_size))
      print("time required: "+str(e_time - s_time))
      return nde
    expanded_nodes = expand(nde)
    number_of_expansion += 1
    tple = tuple(map(tuple, nde.state))
    is_visited.add(tple)

    for child_node in expanded_nodes:
      child_tuple = tuple(map(tuple, child_node.state))
      if(child_tuple not in is_visited):
        q.put(child_node)
    if(q.qsize() > max_queue_size):
      max_queue_size = q.qsize()
  print("queue is empty!")
  return None

In [27]:
state = [[0, 7, 2], [4, 6, 1], [3, 5, 8]]
node = Node(state)
a_star_with_UC(node)

Streaming output truncated to the last 5000 lines.
[[8, 2, 4], [0, 1, 3], [6, 7, 5]] is the best sub-puzzle to be solved with toal cost: g(n) is 7485
[[1, 3, 7], [2, 5, 6], [8, 4, 0]] is the best sub-puzzle to be solved with toal cost: g(n) is 7484
[[7, 3, 2], [4, 1, 8], [0, 5, 6]] is the best sub-puzzle to be solved with toal cost: g(n) is 7484
[[7, 3, 1], [8, 6, 4], [2, 5, 0]] is the best sub-puzzle to be solved with toal cost: g(n) is 7484
[[5, 1, 8], [6, 2, 0], [3, 4, 7]] is the best sub-puzzle to be solved with toal cost: g(n) is 7485
[[7, 3, 0], [8, 6, 1], [2, 5, 4]] is the best sub-puzzle to be solved with toal cost: g(n) is 7484
[[3, 4, 2], [1, 0, 8], [7, 5, 6]] is the best sub-puzzle to be solved with toal cost: g(n) is 7484
[[5, 4, 7], [6, 2, 3], [0, 1, 8]] is the best sub-puzzle to be solved with toal cost: g(n) is 7486
[[5, 4, 1], [3, 7, 6], [0, 8, 2]] is the best sub-puzzle to be solved with toal cost: g(n) is 7486
[[5, 1, 8], [7, 6, 2], [3, 4, 0]] is the best sub-puzzle t

In [29]:
def a_star_with_MD(node):
  q = PriorityQueue()
  q.put(node)
  is_visited = set()
  number_of_expansion = 0
  s_time = time.process_time()
  max_queue_size = 0
  while (q.empty() == False):
    nde = q.get()
    print(nde.state, 'is the best sub-puzzle to be solved with g(n): '+str(nde.g)+ ' and h(n): '+str(nde.h))
    if(misplaced_tile(nde.state) == 0):
      e_time = time.process_time()
      print("reached the goal state!")
      print("the number of expansion required: "+ str(number_of_expansion))
      print("maximum queue size: "+ str(max_queue_size))
      print("time required: "+str(e_time - s_time))
      return nde
    expanded_nodes = expand(nde)
    number_of_expansion += 1
    tple = tuple(map(tuple, nde.state))
    is_visited.add(tple)

    for child_node in expanded_nodes:
      child_tuple = tuple(map(tuple, child_node.state))
      if(child_tuple not in is_visited):
        child_node.h = manhattan_distance(child_node.state)
        child_node.total_cost = child_node.g + child_node.h
        q.put(child_node)
    if(q.qsize() > max_queue_size):
      max_queue_size = q.qsize()
  print("queue is empty!")
  return None

In [30]:
state = [[0, 7, 2], [4, 6, 1], [3, 5, 8]]
node = Node(state)
a_star_with_MD(node)

[[0, 7, 2], [4, 6, 1], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 0 and h(n):0
[[7, 0, 2], [4, 6, 1], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 1 and h(n):13
[[7, 2, 0], [4, 6, 1], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 2 and h(n):12
[[7, 2, 1], [4, 6, 0], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 3 and h(n):11
[[7, 2, 1], [4, 0, 6], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 4 and h(n):10
[[7, 2, 1], [4, 5, 6], [3, 0, 8]] is the best sub-puzzle to be solved with g(n): 5 and h(n):9
[[7, 2, 1], [4, 5, 6], [0, 3, 8]] is the best sub-puzzle to be solved with g(n): 6 and h(n):8
[[7, 2, 1], [4, 5, 6], [3, 8, 0]] is the best sub-puzzle to be solved with g(n): 6 and h(n):8
[[7, 2, 1], [0, 4, 6], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 5 and h(n):11
[[7, 6, 2], [4, 0, 1], [3, 5, 8]] is the best sub-puzzle to be solved with g(n): 2 and h(n):14
[[7, 2, 1], [0, 5, 6], [4, 3, 8]] is the best sub-puzz

In [31]:
def a_star_with_MT(node):
  q = PriorityQueue()
  q.put(node)
  is_visited = set()
  number_of_expansion = 0
  s_time = time.process_time()
  max_queue_size = 0
  while (q.empty() == False):
    nde = q.get()
    print(nde.state, 'is the best sub-puzzle to be solved with g(n): '+str(nde.g)+ ' and h(n): '+str(nde.h))
    if(misplaced_tile(nde.state) == 0):
      e_time = time.process_time()
      print("reached the goal state!")
      print("the number of expansion required: "+ str(number_of_expansion))
      print("maximum queue size: "+ str(max_queue_size))
      print("time required: "+str(e_time - s_time))
      return nde
    expanded_nodes = expand(nde)
    number_of_expansion += 1
    tple = tuple(map(tuple, nde.state))
    is_visited.add(tple)

    for child_node in expanded_nodes:
      child_tuple = tuple(map(tuple, child_node.state))
      if(child_tuple not in is_visited):
        child_node.h = misplaced_tile(child_node.state)
        child_node.total_cost = child_node.g + child_node.h
        q.put(child_node)
    if(q.qsize() > max_queue_size):
      max_queue_size = q.qsize()
  print("queue is empty!")
  return None

In [32]:
state = [[1, 6, 7], [5, 0, 3], [4, 8, 2]]
node = Node(state)
a_star_with_MT(node)

[[1, 6, 7], [5, 0, 3], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 0 and h(n): 0
[[1, 6, 7], [0, 5, 3], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 1 and h(n): 5
[[1, 6, 7], [4, 5, 3], [0, 8, 2]] is the best sub-puzzle to be solved with g(n): 2 and h(n): 4
[[1, 0, 7], [5, 6, 3], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 1 and h(n): 6
[[1, 6, 7], [5, 3, 0], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 1 and h(n): 6
[[1, 6, 7], [5, 8, 3], [4, 0, 2]] is the best sub-puzzle to be solved with g(n): 1 and h(n): 7
[[1, 7, 0], [5, 6, 3], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 2 and h(n): 6
[[1, 6, 0], [5, 3, 7], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 2 and h(n): 6
[[1, 6, 7], [5, 3, 2], [4, 8, 0]] is the best sub-puzzle to be solved with g(n): 2 and h(n): 6
[[1, 7, 3], [5, 6, 0], [4, 8, 2]] is the best sub-puzzle to be solved with g(n): 3 and h(n): 5
[[0, 6, 7], [1, 5, 3], [4, 8, 2]] is the best sub-